In [1]:
from pyspark.sql import *
import getpass
username = getpass.getuser()
spark = SparkSession.builder \
    .appName('loans_defaulter') \
    .config('spark.ui.port', '0') \
    .config('spark.shuffle.useOldFetchProtocol','true') \
    .config("spark.sql.warehouse.dir", f"/user/{username}/warehouse") \
    .enableHiveSupport() \
    .master('yarn') \
    .getOrCreate()

In [2]:
spark.sql("use itv_012667_lending_club")

""


In [3]:
spark.sql("show tables")

database,tableName,isTemporary
itv_012667_lendin...,customer_data,false
itv_012667_lendin...,loan_repayments,false
itv_012667_lendin...,loans,false
itv_012667_lendin...,loans_defaulters_...,false
itv_012667_lendin...,loans_defaulters_...,false


In [4]:
spark.sql("select * from loans")

loand_id,member_id,loan_amount,funded_amount,term,int_rate,installment,issue_d,loan_status,purpose,title,ingestTimestamp
56633077,b59d80da191f5b573...,3000.0,3000.0,3,7.89,93.86,Aug-2015,Fully Paid,credit_card,Credit card refin...,2024-05-26 20:16:...
55927518,202d9f56ecb7c3bc9...,15600.0,15600.0,3,7.89,488.06,Aug-2015,Fully Paid,credit_card,Credit card refin...,2024-05-26 20:16:...
56473345,e5a140c0922b554b9...,20000.0,20000.0,3,9.17,637.58,Aug-2015,Fully Paid,debt_consolidation,Debt consolidation,2024-05-26 20:16:...
56463188,e12aefc548f750777...,11200.0,11200.0,5,21.99,309.27,Aug-2015,Fully Paid,home_improvement,Home improvement,2024-05-26 20:16:...
56473316,1b3a50d854fbbf97e...,16000.0,16000.0,5,20.99,432.77,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2024-05-26 20:16:...
56663266,1c4329e5f17697127...,20000.0,20000.0,5,13.33,458.45,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2024-05-26 20:16:...
56483027,5026c86ad983175eb...,10000.0,10000.0,3,12.69,335.45,Aug-2015,Fully Paid,other,Other,2024-05-26 20:16:...
56613385,9847d8c1e9d0b2084...,23400.0,23400.0,5,19.19,609.46,Aug-2015,Current,small_business,Business,2024-05-26 20:16:...
56643620,8340dbe1adea41fb4...,16000.0,16000.0,3,5.32,481.84,Jul-2015,Fully Paid,debt_consolidation,Debt consolidation,2024-05-26 20:16:...
56533114,d4de0de3ab7d79ad4...,25450.0,25450.0,3,27.31,1043.24,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2024-05-26 20:16:...


In [10]:
spark.sql("""
create or replace view itv005857_lending_club.customers_loan_v as 
select c.member_id,c.address_state,lr.total_payment,d.delinq_2yrs from itv_012667_lending_club.customer_data c
left join loans l on l.member_id = c.member_id
left join loan_repayments lr on lr.loan_id = l.loand_id
left join loans_defaulters_delinq d on c.member_id = d.member_id
left join loans_defaulters_detail_rec_enq e on c.member_id = e.member_id
""")

""


In [11]:
spark.sql("select member_id,total_payment from itv005857_lending_club.customers_loan_v")

member_id,total_payment
000c8875b71a6b47c...,3114.47
000fc98fc1ca5faa3...,4328.2803
0012728d9f616bdf2...,847.62
00151ece27c7ca280...,14992.68
002800d49886390d2...,17558.6
003715c2aec34dd43...,12208.02
003769d7f54c7859e...,267.78998
0037bb910c0a758f5...,343.46
003d7bee408492f11...,9077.4
003e1e6cbd2920bbb...,5950.82
